<a href="https://colab.research.google.com/github/matthew110395/12004210_DataAnalytics/blob/main/12004210_DAOTW_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
This assignment builds on the New York taxi problem identified in assignment 1, where the City of New York is looking for a way to accurately predict the number of collisions on a particular day of the week. Within this document two different types of machine learning models will be utilised to predict the number of collisions. The linear relationships identified in assignment 1 will be used to create a linear regression model. A Deep Learning Neural Network (DNN) will be used to predict the number of collisions where the relationship is not linear. 

#Imports
To prepare data for machine learning the pandas package has been used. The numpy package has been used to aid with mathematical functions.

Within this document, Tensorflow has been used for machine learning, with both a linear regression model and a Deep Neural Network model. Tensorflow version 1 is unsupported within Google Colab, therefore must be installed using the pip package manager.

Shutil has also been imported to allow for easy file management, in particular the removal of saved models.

In [124]:
import pandas as pd
import numpy as np

!pip install tensorflow==1.15.2
import tensorflow as tf
# needed for high-level file management
import shutil  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Linear Regressor


##Precipition

In [125]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/prcp_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [126]:
df_prcp = df.drop(columns=['collision_date', 'temp', 'dewp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud'])
df_prcp = df_prcp.loc[df_prcp["year"] != 2012]
df_prcp = df_prcp.loc[df_prcp["year"] < 2020]
cols = df_prcp['NUM_COLLISIONS']
df_prcp = df_prcp.drop(columns=['NUM_COLLISIONS'])
df_prcp.insert(loc=9, column='NUM_COLLISIONS', value=cols)
print(df_prcp[:6])
df_prcp.describe()

    day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  \
49    4  2016   1  28  0.09    0             0                 0     0   
51    5  2014   1  17  0.00    1             0                 0     0   
54    1  2016   1  25  0.02    0             0                 0     0   
55    5  2016   1  29  0.00    0             0                 0     0   
58    5  2017   1  20  0.00    0             0                 0     0   
59    7  2013   1  13  0.01    1             0                 0     0   

    NUM_COLLISIONS  
49             681  
51             589  
54             658  
55             645  
58             605  
59             373  


,day,year,mo,da,prcp,fog,rain_drizzle,snow_ice_pellets,hail,NUM_COLLISIONS
count,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000
mean,3.998425,2015.989366,6.518708,15.745569,0.122588,0.253249,0.375345,0.085467,0.000394,599.135093
std,2.003542,1.996126,3.455211,8.803199,0.329143,0.434958,0.484307,0.279630,0.019846,100.299164
min,1.000000,2013.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,0.060000,1.000000,1.000000,0.000000,0.000000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,3.760000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [127]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_prcp.iloc[np.random.permutation(len(df_prcp))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail
1580    1  2013   6  10  0.00    0             1                 0     0
668     3  2018   3  28  0.00    0             0                 0     0
1067    2  2016   4  12  0.00    0             1                 0     0
3359    6  2018  11  10  1.32    0             1                 0     0
390     4  2015   2  12  0.00    0             0                 1     0
3547    6  2016  12  10  0.00    0             0                 0     0
      day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  \
1580    1  2013   6  10  0.00    0             1                 0     0   
668     3  2018   3  28  0.00    0             0                 0     0   
1067    2  2016   4  12  0.00    0             1                 0     0   
3359    6  2018  11  10  1.32    0             1                 0     0   
390     4  2015   2  12  0.00    0             0                 1     0   
3547    6  2016  12  10  0.00    

In [128]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1580    624
668     629
1067    638
3359    606
390     526
3547    555
Name: NUM_COLLISIONS, dtype: int64


In [129]:
SCALE_COLLISIONS=1161

In [130]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 9
# Define the number of output values (targets)
noutputs = 1

2031
508


In [131]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_prcp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_prcp', optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77d784e590>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_prcp/model.ckpt.
INFO:tensorflow:loss = 0.2741817, step = 1
INFO:tensorflow:global_step/sec: 531.573
INFO:tensorflow:loss = 0.008613801, step = 101 (0.196 sec)
INFO:tensorflow:global_step/sec: 552.764
INFO:tensorflow:loss = 0.007745385, step = 201 (0.180 sec)
INFO:tensorflow:global_step/sec: 751.437
INFO:tensorflow:loss = 0.0065500243, step = 301 (0.130 sec)
INFO:tensorflow:global_step/sec: 595.845
INFO:tensorflow:loss = 0.0060137557, step = 401 (0.173 sec)
INFO:tensorflow:global_step/sec: 585.097
INFO:tensorflow:loss = 0.00856616, step = 501 (0.170 sec)
INFO:tensorflow:global_step/sec: 628.276
INFO:tensorflow:loss = 0.007597821, step = 601 (0.156 sec)
INFO:tensorflow:global_step/sec: 875.888
INFO:tensorflow:loss = 0.006424806, step = 701 (0.1

LinearRegression has RMSE of 133.62876330148148
Just using average = 600.0236336779911 has RMSE of 99.74965071762696


In [132]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_prcp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77d3020890>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_prc

508
508
[0.42987642 0.39094666 0.44640788 0.45076153 0.4303333  0.4814665
 0.4429572  0.39470223 0.42254275 0.47868174 0.376789   0.38981938
 0.40248543 0.43961838 0.45967454 0.4122563  0.4241293  0.42008883
 0.37994906 0.46601748 0.4690539  0.42026335 0.48523942 0.4616028
 0.44085035 0.4030289  0.38974577 0.41590118 0.3750105  0.38319585
 0.41171265 0.43872818 0.45940155 0.4285984  0.40527162 0.41341102
 0.46074495 0.48642397 0.41700634 0.4639533  0.43161786 0.39841288
 0.44786492 0.4059397  0.37056044 0.40497023 0.43759826 0.4126105
 0.44341254 0.4724253  0.44972336 0.48057538 0.46801826 0.38432828
 0.3864209  0.42413628 0.42837426 0.44011092 0.44224393 0.41302282
 0.40347815 0.4218455  0.4394321  0.47718993 0.40395692 0.41018307
 0.42806706 0.40087438 0.42380098 0.38031417 0.44316158 0.4023234
 0.4158103  0.3762061  0.3977744  0.3847511  0.43397504 0.40719318
 0.39132127 0.4186721  0.44128954 0.48102093 0.39237988 0.48179236
 0.40436926 0.45717964 0.46818137 0.45707345 0.41133514 0.

##Dew Point (dewp)


In [133]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/dewp_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [134]:
df_dewp = df.drop(columns=['collision_date', 'temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_dewp = df_dewp.loc[df_dewp["year"] != 2012]
df_dewp = df_dewp.loc[df_dewp["year"] < 2020]
cols = df_dewp['NUM_COLLISIONS']
df_dewp = df_dewp.drop(columns=['NUM_COLLISIONS'])
df_dewp.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_dewp[:6])
df_dewp.describe()

    day  year  mo  da  dewp  NUM_COLLISIONS
49    4  2016   1  28  24.4             681
51    5  2014   1  17  35.8             589
54    1  2016   1  25  21.2             658
55    5  2016   1  29  36.8             645
58    5  2017   1  20  32.5             605
59    7  2013   1  13  44.9             373


,day,year,mo,da,dewp,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,3.998434,2015.999217,6.524070,15.723679,44.163170,599.109980
std,2.000391,2.000000,3.449676,8.801271,16.995303,100.277185
min,1.000000,2013.000000,1.000000,1.000000,-6.700000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,32.150000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,45.300000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,58.500000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,74.100000,1161.000000


In [135]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_dewp.iloc[np.random.permutation(len(df_dewp))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  dewp
3205    2  2018  11   6  56.2
1032    4  2014   4  10  28.9
1244    7  2014   5  25  44.4
2048    3  2015   7   8  67.8
3541    2  2013  12  17  15.9
2533    3  2018   9  12  69.6
      day  year  mo  da  dewp  NUM_COLLISIONS
3205    2  2018  11   6  56.2             679
1032    4  2014   4  10  28.9             645
1244    7  2014   5  25  44.4             472
2048    3  2015   7   8  67.8             620
3541    2  2013  12  17  15.9             703
2533    3  2018   9  12  69.6             783


In [136]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

3205    679
1032    645
1244    472
2048    620
3541    703
2533    783
Name: NUM_COLLISIONS, dtype: int64


In [137]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

2044
511


In [138]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_dewp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_dewp', optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77d2e5a310>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_dewp/model.ckpt.
INFO:tensorflow:loss = 0.27211958, step = 1
INFO:tensorflow:global_step/sec: 745.714
INFO:tensorflow:loss = 0.007127203, step = 101 (0.137 sec)
INFO:tensorflow:global_step/sec: 886.097
INFO:tensorflow:loss = 0.006825781, step = 201 (0.114 sec)
INFO:tensorflow:global_step/sec: 843.359
INFO:tensorflow:loss = 0.005263289, step = 301 (0.119 sec)
INFO:tensorflow:global_step/sec: 915.963
INFO:tensorflow:loss = 0.0076429676, step = 401 (0.107 sec)
INFO:tensorflow:global_step/sec: 818.938
INFO:tensorflow:loss = 0.007173229, step = 501 (0.124 sec)
INFO:tensorflow:global_step/sec: 617.845
INFO:tensorflow:loss = 0.0051976102, step = 601 (0.161 sec)
INFO:tensorflow:global_step/sec: 775.507
INFO:tensorflow:loss = 0.0059868535, step = 701 (

LinearRegression has RMSE of 101.39611547216715
Just using average = 599.1081213307241 has RMSE of 107.18799439834025


In [139]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_dewp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77ceeac090>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_dew

511
511
[0.49113408 0.4881626  0.5006525  0.50036126 0.5045505  0.50081605
 0.5428108  0.47471237 0.5205801  0.49932688 0.5012491  0.54813516
 0.53094643 0.5179871  0.52326894 0.47981945 0.5285742  0.48713955
 0.49731293 0.4614843  0.47878057 0.5031496  0.50030917 0.5063216
 0.47203416 0.4492744  0.52781445 0.46927723 0.4637366  0.50760883
 0.4437548  0.5266648  0.49465743 0.472608   0.48636907 0.542532
 0.5282972  0.55163616 0.52802694 0.52605313 0.5206262  0.51341325
 0.52215505 0.46828103 0.5454357  0.47908694 0.47307602 0.45605692
 0.47889918 0.53389543 0.5324805  0.47342047 0.52959937 0.519376
 0.5062903  0.49052638 0.4529709  0.48924968 0.511345   0.49358004
 0.43346342 0.48982152 0.500176   0.48429143 0.48101178 0.44719625
 0.46459624 0.5128059  0.4620526  0.53577125 0.5417075  0.41498902
 0.4960257  0.5297887  0.5505703  0.5121371  0.4342537  0.5345716
 0.51635736 0.48388025 0.48215672 0.49695295 0.5048197  0.5514952
 0.48294827 0.4798197  0.48168457 0.44222897 0.46128792 0.490

##Sea Level Pressure (SLP)


In [140]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/slp_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [141]:
df_slp = df.drop(columns=['collision_date', 'temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_slp = df_slp.loc[df_slp["year"] != 2012]
df_slp = df_slp.loc[df_slp["year"] < 2020]
cols = df_slp['NUM_COLLISIONS']
df_slp = df_slp.drop(columns=['NUM_COLLISIONS'])
df_slp.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_slp[:6])
df_slp.describe()

    day  year  mo  da     slp  NUM_COLLISIONS
49    4  2016   1  28  1016.1             681
51    5  2014   1  17  1014.8             589
54    1  2016   1  25  1021.4             658
55    5  2016   1  29   999.4             645
58    5  2017   1  20  1015.5             605
59    7  2013   1  13  1020.7             373


,day,year,mo,da,slp,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,3.999217,2016.000391,6.522114,15.719765,1016.777221,599.147162
std,2.000783,2.000294,3.447986,8.796698,7.628429,100.268048
min,1.000000,2013.000000,1.000000,1.000000,989.500000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,1012.200000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,1016.700000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,1021.700000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,1044.200000,1161.000000


In [142]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_slp.iloc[np.random.permutation(len(df_slp))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da     slp
1594    1  2014   6  23  1019.7
505     4  2016   2   4  1015.2
288     2  2014   1   7  1009.6
2347    2  2013   8  27  1011.8
469     2  2018   2  13  1039.1
1243    4  2018   5  17  1016.7
      day  year  mo  da     slp  NUM_COLLISIONS
1594    1  2014   6  23  1019.7             644
505     4  2016   2   4  1015.2             609
288     2  2014   1   7  1009.6             491
2347    2  2013   8  27  1011.8             506
469     2  2018   2  13  1039.1             613
1243    4  2018   5  17  1016.7             732


In [143]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1594    644
505     609
288     491
2347    506
469     613
1243    732
Name: NUM_COLLISIONS, dtype: int64


In [144]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

2044
511


In [145]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_slp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_slp', optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77d2c17f50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_slp/model.ckpt.
INFO:tensorflow:loss = 0.2822011, step = 1
INFO:tensorflow:global_step/sec: 664.653
INFO:tensorflow:loss = 0.0069908286, step = 101 (0.156 sec)
INFO:tensorflow:global_step/sec: 846.698
INFO:tensorflow:loss = 0.0058898525, step = 201 (0.118 sec)
INFO:tensorflow:global_step/sec: 834.234
INFO:tensorflow:loss = 0.008527931, step = 301 (0.118 sec)
INFO:tensorflow:global_step/sec: 912.96
INFO:tensorflow:loss = 0.007766745, step = 401 (0.111 sec)
INFO:tensorflow:global_step/sec: 865.974
INFO:tensorflow:loss = 0.005420659, step = 501 (0.113 sec)
INFO:tensorflow:global_step/sec: 846.156
INFO:tensorflow:loss = 0.008386809, step = 601 (0.117 sec)
INFO:tensorflow:global_step/sec: 845.098
INFO:tensorflow:loss = 0.008839803, step = 701 (0.11

LinearRegression has RMSE of 99.57043038931194
Just using average = 599.0053816046967 has RMSE of 101.6203275486354


In [146]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_slp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77d2f1fe10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_slp/

511
511
[0.52776057 0.55039567 0.56388557 0.56103015 0.5632473  0.5694784
 0.56920135 0.54858345 0.49145693 0.60125995 0.58724165 0.58848876
 0.57424855 0.5052264  0.5344421  0.59125817 0.5666763  0.51085484
 0.5052988  0.5773522  0.60142255 0.5502752  0.55396324 0.55439365
 0.5511964  0.5029115  0.5738866  0.53181314 0.5552395  0.5701349
 0.5732831  0.5835354  0.53424156 0.5264206  0.57829803 0.5454287
 0.53141344 0.5630133  0.5320521  0.57594097 0.5217124  0.58564687
 0.5332876  0.51614517 0.5094164  0.5226072  0.5006325  0.5035109
 0.5768346  0.5375358  0.5490387  0.54216015 0.5261327  0.5263965
 0.57459825 0.5227212  0.5118114  0.555989   0.5264751  0.54808635
 0.5487294  0.5327456  0.53229076 0.53256637 0.55798393 0.5194811
 0.5191365  0.4913669  0.5168462  0.5344946  0.53740305 0.5645722
 0.5820208  0.5451066  0.5416231  0.55108875 0.525167   0.52962786
 0.5480791  0.5003842  0.56116617 0.53869003 0.50990415 0.5227779
 0.5522162  0.53717    0.52465844 0.5931069  0.57737666 0.5511

##Gust

In [147]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/gust_clean.csv', index_col=0, )
print(df[:6])

    day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
3     5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
11    3  2020   1  15     2020-01-15             508  43.9  38.3  1019.4   
12    5  2021   1   1     2021-01-01             257  39.6  29.3  1029.3   
14    2  2022   1  25     2022-01-25             235  41.6  31.8  1013.2   
18    7  2021   1   3     2021-01-03             186  41.1  32.3  1018.0   
19    4  2020   1   2     2020-01-02             413  39.6  28.9  1011.8   

    visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
3    10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
11    8.2  ...  51.1  35.1  0.02  999.9    1             0                 0   
12   10.0  ...  54.0  33.1  0.13  999.9    0             0                 0   
14   10.0  ...  48.9  30.0  0.00  999.9    0             0                 0   
18   10.0  ...  53.1  39.0  0.00  999.9    0             0         

In [148]:
df_gust = df.drop(columns=['collision_date', 'temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','dewp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_gust = df_gust.loc[df_gust["year"] != 2012]
df_gust = df_gust.loc[df_gust["year"] < 2020]
cols = df_gust['NUM_COLLISIONS']
df_gust = df_gust.drop(columns=['NUM_COLLISIONS'])
df_gust.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_gust[:6])
df_gust.describe()

    day  year  mo  da  gust  NUM_COLLISIONS
74    7  2016   1  17  18.1             451
76    4  2014   1   9  20.0             561
79    6  2019   1  19  21.0             479
80    7  2015   1  11  17.1             341
83    4  2015   1  29  20.0             519
85    7  2019   1  13  15.9             374


,day,year,mo,da,gust,NUM_COLLISIONS
count,1629.000000,1629.00000,1629.000000,1629.000000,1629.000000,1629.000000
mean,4.024555,2015.91283,6.278699,15.702885,27.511602,596.513198
std,1.989070,2.01341,3.747683,8.667634,7.366770,104.479660
min,1.000000,2013.00000,1.000000,1.000000,14.000000,188.000000
25%,2.000000,2014.00000,3.000000,8.000000,22.000000,526.000000
50%,4.000000,2016.00000,6.000000,16.000000,26.000000,597.000000
75%,6.000000,2018.00000,10.000000,23.000000,31.100000,663.000000
max,7.000000,2019.00000,12.000000,31.000000,71.100000,1161.000000


In [149]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_gust.iloc[np.random.permutation(len(df_gust))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  gust
2133    4  2013   7  25  27.0
3256    3  2017  11  29  26.0
3544    6  2014  12   6  25.1
1116    6  2017   4  29  28.0
3169    2  2017  11  28  22.0
174     1  2017   1   9  22.9
      day  year  mo  da  gust  NUM_COLLISIONS
2133    4  2013   7  25  27.0             507
3256    3  2017  11  29  26.0             701
3544    6  2014  12   6  25.1             628
1116    6  2017   4  29  28.0             661
3169    2  2017  11  28  22.0             649
174     1  2017   1   9  22.9             669


In [150]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

2133    507
3256    701
3544    628
1116    661
3169    649
174     669
Name: NUM_COLLISIONS, dtype: int64


In [151]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

1303
326


In [152]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_gust', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_gust', optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77d2c4a850>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_gust/model.ckpt.
INFO:tensorflow:loss = 0.27536982, step = 1
INFO:tensorflow:global_step/sec: 794.082
INFO:tensorflow:loss = 0.00824094, step = 101 (0.132 sec)
INFO:tensorflow:global_step/sec: 903.256
INFO:tensorflow:loss = 0.0077048824, step = 201 (0.109 sec)
INFO:tensorflow:global_step/sec: 868.591
INFO:tensorflow:loss = 0.008711301, step = 301 (0.114 sec)
INFO:tensorflow:global_step/sec: 848.131
INFO:tensorflow:loss = 0.007413054, step = 401 (0.120 sec)
INFO:tensorflow:global_step/sec: 888.238
INFO:tensorflow:loss = 0.00823536, step = 501 (0.112 sec)
INFO:tensorflow:global_step/sec: 879.486
INFO:tensorflow:loss = 0.0076253586, step = 601 (0.116 sec)
INFO:tensorflow:global_step/sec: 887.428
INFO:tensorflow:loss = 0.00986084, step = 701 (0.11

LinearRegression has RMSE of 92.94966471518207
Just using average = 596.8211818879508 has RMSE of 103.00548183434161


In [153]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_gust', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77cec96cd0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


326
326


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_gust/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.5557709  0.46615773 0.55753773 0.46943542 0.50679266 0.512485
 0.580244   0.5260611  0.4998476  0.46684396 0.5563261  0.53378344
 0.5423882  0.54754436 0.48671144 0.50661725 0.4632599  0.47780636
 0.47252205 0.49141082 0.5076127  0.5468842  0.4914178  0.5118759
 0.48875952 0.5689169  0.45469388 0.55350274 0.5143688  0.5031826
 0.48916844 0.5315987  0.4908127  0.5207861  0.55147976 0.55261445
 0.48084858 0.48353618 0.54467714 0.56474954 0.47844648 0.55962193
 0.5180613  0.58144546 0.51866114 0.48401842 0.5406124  0.46837023
 0.53333557 0.56471014 0.51895875 0.54937166 0.47446012 0.52627575
 0.50368583 0.53781855 0.45744017 0.5340738  0.45110118 0.46662527
 0.47060412 0.4848266  0.5701507  0.5213772  0.52565205 0.49091944
 0.543196   0.4990372  0.5575727  0.4920895  0.4823263  0.50947297
 0.5409498  0.54177517 0.4924738  0.454851   0.48959112 0.5066214
 0.5067871  0.4976507  0.5220085  0.5598408  0.5522935  0.45103437
 0.5076723  0.46354663 0.52495784 0.5282798  0.5630474  0.53658473


##Maximum Sustained Wind Speed (Mxpsd)

In [154]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/mxpsd_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [155]:
df_mxpsd = df.drop(columns=['collision_date', 'temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','gust','dewp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_mxpsd = df_mxpsd.loc[df_mxpsd["year"] != 2012]
df_mxpsd = df_mxpsd.loc[df_mxpsd["year"] < 2020]
cols = df_mxpsd['NUM_COLLISIONS']
df_mxpsd = df_mxpsd.drop(columns=['NUM_COLLISIONS'])
df_mxpsd.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_mxpsd[:6])
df_mxpsd.describe()

    day  year  mo  da  mxpsd  NUM_COLLISIONS
49    4  2016   1  28    8.9             681
51    5  2014   1  17    8.9             589
54    1  2016   1  25    8.9             658
55    5  2016   1  29    9.9             645
58    5  2017   1  20    9.9             605
59    7  2013   1  13    9.9             373


,day,year,mo,da,mxpsd,NUM_COLLISIONS
count,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000
mean,3.999608,2016.001567,6.520564,15.737172,17.240110,599.033686
std,2.001469,2.000587,3.449204,8.797367,5.858333,100.284761
min,1.000000,2013.000000,1.000000,1.000000,5.100000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,13.000000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,15.900000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,20.000000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,49.000000,1161.000000


In [156]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_mxpsd.iloc[np.random.permutation(len(df_mxpsd))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  mxpsd
441     7  2017   2  19   17.1
2967    1  2017  10   9   18.1
1957    1  2017   7  17   11.1
1595    6  2016   6   4    9.9
2942    7  2014  10   5   17.1
1645    1  2014   6  30   12.0
      day  year  mo  da  mxpsd  NUM_COLLISIONS
441     7  2017   2  19   17.1             438
2967    1  2017  10   9   18.1             546
1957    1  2017   7  17   11.1             639
1595    6  2016   6   4    9.9             635
2942    7  2014  10   5   17.1             505
1645    1  2014   6  30   12.0             610


In [157]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

441     438
2967    546
1957    639
1595    635
2942    505
1645    610
Name: NUM_COLLISIONS, dtype: int64


In [158]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

2042
511


In [159]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_mxpsd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_mxpsd', optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77cee99550>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_mxpsd', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_mxpsd/model.ckpt.
INFO:tensorflow:loss = 0.27496588, step = 1
INFO:tensorflow:global_step/sec: 707.813
INFO:tensorflow:loss = 0.0067426204, step = 101 (0.148 sec)
INFO:tensorflow:global_step/sec: 826.68
INFO:tensorflow:loss = 0.006906179, step = 201 (0.120 sec)
INFO:tensorflow:global_step/sec: 770.618
INFO:tensorflow:loss = 0.008527726, step = 301 (0.131 sec)
INFO:tensorflow:global_step/sec: 815.18
INFO:tensorflow:loss = 0.0063469727, step = 401 (0.119 sec)
INFO:tensorflow:global_step/sec: 867.903
INFO:tensorflow:loss = 0.008620524, step = 501 (0.119 sec)
INFO:tensorflow:global_step/sec: 709.404
INFO:tensorflow:loss = 0.008293173, step = 601 (0.138 sec)
INFO:tensorflow:global_step/sec: 870.981
INFO:tensorflow:loss = 0.0061335806, step = 701 (0

LinearRegression has RMSE of 101.52706958809091
Just using average = 599.2531831537708 has RMSE of 105.32995403460292


In [160]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_mxpsd', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77e789d7d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_mxpsd', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_mx

511
511
[0.5293082  0.45215654 0.5274805  0.5408062  0.5190872  0.5424172
 0.51296085 0.5314365  0.5083273  0.45073065 0.47601122 0.47475526
 0.55633026 0.4968665  0.5222989  0.50373894 0.48763293 0.49780604
 0.47671187 0.4903994  0.53298473 0.5074837  0.53918684 0.5596663
 0.486301   0.49028987 0.5306759  0.51404196 0.51402485 0.5284784
 0.5108726  0.49801183 0.5377447  0.48471156 0.4934774  0.50311685
 0.4780389  0.46411392 0.45845762 0.48775643 0.49416402 0.52164763
 0.5230926  0.50950664 0.52390456 0.4723207  0.49155954 0.48216158
 0.47403118 0.4793328  0.47140902 0.5235308  0.50898224 0.49873555
 0.53903866 0.49509123 0.5201238  0.5538446  0.46266568 0.496634
 0.52019906 0.5249832  0.5053259  0.52857107 0.46512175 0.48035
 0.52722013 0.4559736  0.47541964 0.53982013 0.48098582 0.49597365
 0.53846496 0.48954016 0.45195374 0.4956513  0.47107282 0.52475154
 0.55246025 0.52242595 0.5120174  0.5077789  0.48453414 0.5286889
 0.5011728  0.44837517 0.52174324 0.52493167 0.5374102  0.48323

##Others


#Deep Learning Neural Network (DNN)

####Precipition (prcp)

In [161]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/prcp_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [162]:
df_prcp_dnn = df.drop(columns=['temp', 'dewp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud'])
df_prcp_dnn = df_prcp_dnn.loc[df_prcp_dnn["year"] != 2012]
df_prcp_dnn = df_prcp_dnn.loc[df_prcp_dnn["year"] < 2020]
cols = df_prcp_dnn['NUM_COLLISIONS']
df_prcp_dnn = df_prcp_dnn.drop(columns=['NUM_COLLISIONS'])
df_prcp_dnn.insert(loc=25, column='NUM_COLLISIONS', value=cols)
print(df_prcp_dnn[:6])
df_prcp_dnn.describe()

    year  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  Apr  Aug  Dec  \
49  2016  28  0.09    0             0                 0     0    0    0    0   
51  2014  17  0.00    1             0                 0     0    0    0    0   
54  2016  25  0.02    0             0                 0     0    0    0    0   
55  2016  29  0.00    0             0                 0     0    0    0    0   
58  2017  20  0.00    0             0                 0     0    0    0    0   
59  2013  13  0.01    1             0                 0     0    0    0    0   

    ...  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49  ...    0    0    0    0    0    0    0    0    1             681  
51  ...    0    0    0    0    0    0    1    0    0             589  
54  ...    0    0    0    0    0    1    0    0    0             658  
55  ...    0    0    0    0    0    0    1    0    0             645  
58  ...    0    0    0    0    0    0    1    0    0             605  
59  ...    0 

,year,da,prcp,fog,rain_drizzle,snow_ice_pellets,hail,Apr,Aug,Dec,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,...,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000
mean,2015.989366,15.745569,0.122588,0.253249,0.375345,0.085467,0.000394,0.082316,0.083497,0.085467,...,0.082710,0.085467,0.079953,0.142970,0.143364,0.143757,0.142182,0.142576,0.142182,599.135093
std,1.996126,8.803199,0.329143,0.434958,0.484307,0.279630,0.019846,0.274899,0.276687,0.279630,...,0.275497,0.279630,0.271273,0.350111,0.350512,0.350913,0.349305,0.349709,0.349305,100.299164
min,2013.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,0.060000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,3.760000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [163]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_prcp_dnn.iloc[np.random.permutation(len(df_prcp_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  Apr  Aug  \
3239  2016  15  0.00    0             1                 0     0    0    0   
3223  2014  14  0.22    0             1                 0     0    0    0   
132   2018  25  0.00    0             0                 0     0    0    0   
223   2013  25  0.00    0             0                 0     0    0    0   
3304  2015   6  0.14    1             1                 0     0    0    0   
2858  2014   6  0.00    0             0                 0     0    0    0   

      Dec  ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
3239    0  ...    0    1    0    0    1    0    0    0    0    0  
3223    0  ...    0    1    0    0    0    0    0    1    0    0  
132     0  ...    0    0    0    0    0    0    0    0    0    1  
223     0  ...    0    0    0    0    0    0    0    1    0    0  
3304    0  ...    0    1    0    0    0    0    0    1    0    0  
2858    0  ...    0    0    1    0    0    0    1    0    

In [164]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

3239    741
3223    654
132     602
223     674
3304    616
2858    639
Name: NUM_COLLISIONS, dtype: int64


In [165]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

25


In [175]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_prcp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_prcp', hidden_units=[20,18,13], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77d7a2a590>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_prcp/model.ckpt.
INFO:tensorflow:loss = 8614.432, step = 1
INFO:tensorflow:global_step/sec: 468.388
INFO:tensorflow:loss = 0.06773439, step = 101 (0.220 sec)
INFO:tensorflow:global_step/sec: 645.575
INFO:tensorflow:loss = 0.046054713, step = 201 (0.154 sec)
INFO:tensorflow:global_step/sec: 576.734
INFO:tensorflow:loss = 0.030378692, step = 301 (0.174 sec)
INFO:tensorflow:global_step/sec: 635.202
INFO:tensorflow:loss = 0.01904181, step = 401 (0.157 sec)
INFO:tensorflow:global_step/sec: 619.223
INFO:tensorflow:loss = 0.013624711, step = 501 (0.163 sec)
INFO:tensorflow:global_step/sec: 628.021
INFO:tensorflow:loss = 0.012396911, step = 601 (0.157 sec)
INFO:tensorflow:global_step/sec: 587.162
INFO:tensorflow:loss = 0.008858361, step = 701 (0.171 sec)

DNNRegression has RMSE of 80.16418117461276
Just using average = 600.313638601674 has RMSE of 98.09899836406518


In [176]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_prcp', hidden_units=[20,18,13], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77d74c3ad0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}


508
508


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_prcp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.53202736 0.44927192 0.5047906  0.59436613 0.467834   0.47121316
 0.48251694 0.5731366  0.5438848  0.53789884 0.5975057  0.5458243
 0.510034   0.43512762 0.49221745 0.53598803 0.5023608  0.55658
 0.47223178 0.44824737 0.47996423 0.5489555  0.45635927 0.59668696
 0.503149   0.52206355 0.5676175  0.40022203 0.554863   0.55872226
 0.4613889  0.5138441  0.49740174 0.51578164 0.51450515 0.5533862
 0.54902065 0.531412   0.45461357 0.566634   0.4943463  0.46970457
 0.5407769  0.56227213 0.5535639  0.5587907  0.5188254  0.5568911
 0.53676575 0.5904612  0.5449157  0.5839668  0.54432863 0.5513478
 0.50105643 0.4920383  0.4117169  0.5811577  0.52079606 0.5106035
 0.54925734 0.5553905  0.5509704  0.63057345 0.54948807 0.5078773
 0.5663043  0.5840157  0.51808244 0.48944616 0.46527946 0.50407535
 0.5640226  0.5703889  0.55228263 0.5783069  0.45676404 0.501073
 0.5144739  0.5235623  0.5108188  0.55908084 0.5510254  0.52196366
 0.5790218  0.5383944  0.52452874 0.58626074 0.5585373  0.53522164
 0.516

##Dew Point (dewp)


In [177]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/dewp_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [182]:
df_dewp_dnn = df.drop(columns=['temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_dewp_dnn = df_dewp_dnn.loc[df_dewp_dnn["year"] != 2012]
df_dewp_dnn = df_dewp_dnn.loc[df_dewp_dnn["year"] < 2020]
cols = df_dewp_dnn['NUM_COLLISIONS']
df_dewp_dnn = df_dewp_dnn.drop(columns=['NUM_COLLISIONS'])
df_dewp_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_dewp_dnn[:6])
df_dewp_dnn.describe()

    year  da  dewp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
49  2016  28  24.4    0    0    0    0    1    0    0  ...    0    0    0   
51  2014  17  35.8    0    0    0    0    1    0    0  ...    0    0    0   
54  2016  25  21.2    0    0    0    0    1    0    0  ...    0    0    0   
55  2016  29  36.8    0    0    0    0    1    0    0  ...    0    0    0   
58  2017  20  32.5    0    0    0    0    1    0    0  ...    0    0    0   
59  2013  13  44.9    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    1             681  
51    0    0    0    1    0    0             589  
54    0    0    1    0    0    0             658  
55    0    0    0    1    0    0             645  
58    0    0    0    1    0    0             605  
59    0    1    0    0    0    0             373  

[6 rows x 22 columns]


,year,da,dewp,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,...,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,2015.999217,15.723679,44.163170,0.082192,0.084932,0.084932,0.077104,0.084932,0.084540,0.082192,...,0.082192,0.084932,0.082192,0.143249,0.142857,0.142857,0.142857,0.142857,0.142857,599.109980
std,2.000000,8.801271,16.995303,0.274710,0.278834,0.278834,0.266808,0.278834,0.278251,0.274710,...,0.274710,0.278834,0.274710,0.350395,0.349996,0.349996,0.349996,0.349996,0.349996,100.277185
min,2013.000000,1.000000,-6.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,32.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,45.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,58.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,74.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [183]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_dewp_dnn.iloc[np.random.permutation(len(df_dewp_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  dewp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
2514  2013  25  45.3    0    0    0    0    0    0    0  ...    0    0    0   
2622  2019  16  58.5    0    0    0    0    0    0    0  ...    0    0    0   
1104  2017  22  44.5    1    0    0    0    0    0    0  ...    0    0    0   
3107  2017  25  47.2    0    0    0    0    0    0    0  ...    0    1    0   
2589  2016   9  68.3    0    0    0    0    0    0    0  ...    0    0    0   
2518  2018  30  53.5    0    0    0    0    0    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
2514    1    0    0    0    0    1    0  
2622    1    0    0    1    0    0    0  
1104    0    0    0    0    0    0    0  
3107    0    0    0    0    0    0    0  
2589    1    0    0    0    1    0    0  
2518    1    0    1    0    0    0    0  

[6 rows x 21 columns]


In [184]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

2514    539
2622    620
1104    551
3107    456
2589    715
2518    510
Name: NUM_COLLISIONS, dtype: int64


In [185]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [190]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_dewp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_dewp', hidden_units=[20,18,13], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77d7a19750>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_dewp/model.ckpt.
INFO:tensorflow:loss = 32075.318, step = 1
INFO:tensorflow:global_step/sec: 469.223
INFO:tensorflow:loss = 0.040787622, step = 101 (0.216 sec)
INFO:tensorflow:global_step/sec: 632.594
INFO:tensorflow:loss = 0.055134676, step = 201 (0.160 sec)
INFO:tensorflow:global_step/sec: 636.53
INFO:tensorflow:loss = 0.033296518, step = 301 (0.154 sec)
INFO:tensorflow:global_step/sec: 597.356
INFO:tensorflow:loss = 0.036332756, step = 401 (0.170 sec)
INFO:tensorflow:global_step/sec: 597.563
INFO:tensorflow:loss = 0.027359705, step = 501 (0.165 sec)
INFO:tensorflow:global_step/sec: 629.818
INFO:tensorflow:loss = 0.024996342, step = 601 (0.159 sec)
INFO:tensorflow:global_step/sec: 578.209
INFO:tensorflow:loss = 0.020741882, step = 701 (0.175 se

DNNRegression has RMSE of 81.82060903868319
Just using average = 600.6712328767123 has RMSE of 99.44895988564187


In [191]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_dewp', hidden_units=[20,18,13], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77e6cf6dd0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}


511
511


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_dewp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.4684564  0.44927323 0.47069657 0.46099484 0.48245823 0.40897667
 0.50621045 0.46217644 0.4745742  0.4928733  0.48814213 0.5281583
 0.5031606  0.5651351  0.48149502 0.5063163  0.5367156  0.49897206
 0.39752305 0.5558244  0.4252435  0.48009408 0.52517235 0.5220634
 0.5132257  0.5206796  0.52939045 0.49740136 0.41314328 0.47809517
 0.45336258 0.4983865  0.545889   0.5651723  0.5377675  0.5540229
 0.5008346  0.45196736 0.5725223  0.49669373 0.51260865 0.49270642
 0.4686042  0.4556123  0.42840683 0.53765786 0.5647088  0.4490615
 0.47431767 0.4909488  0.53751004 0.4324571  0.53868973 0.4546758
 0.41799843 0.5105077  0.36416352 0.39913857 0.5099584  0.45390332
 0.51978505 0.4897548  0.4223863  0.49297917 0.5541154  0.40536702
 0.46667016 0.45212662 0.47995102 0.45412457 0.5657712  0.53416836
 0.37718785 0.55272305 0.36525452 0.3874904  0.4446317  0.4991504
 0.5447093  0.51048577 0.53260624 0.51708233 0.4323951  0.4336568
 0.52571213 0.44959462 0.41206563 0.5014945  0.54465973 0.538087
 0.4